In [ ]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

import category_encoders as ce
from feature_engineering import encoding


## Load Dataset

In [ ]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]

data = pd.read_csv('https://raw.githubusercontent.com/Yimeng-Zhang/feature-engineering-and-feature-selection/master/data/titanic.csv', usecols=use_cols)
data.head()

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,male,22.0,1,7.2500
1,1,1,female,38.0,1,71.2833
2,1,3,female,26.0,0,7.9250
3,1,1,female,35.0,1,53.1000
4,0,3,male,35.0,0,8.0500


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, data.Survived, test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((623, 6), (268, 6))

## One-hot encoding
replace the categorical variable by different boolean variables (0/1) to indicate whether or not certain label is true for that observation

In [ ]:
data1 = pd.get_dummies(data,drop_first=True)

In [ ]:
data1.head()

,Survived,Pclass,Age,SibSp,Fare,Sex_male
0,0,3,22.0,1,7.2500,1
1,1,1,38.0,1,71.2833,0
2,1,3,26.0,0,7.9250,0
3,1,1,35.0,1,53.1000,0
4,0,3,35.0,0,8.0500,1


## Ordinal-encoding
replace the labels by some ordinal number if ordinal is meaningful

In [ ]:
ord_enc = ce.OrdinalEncoder(cols=['Sex']).fit(X_train,y_train)

In [ ]:
data4 = ord_enc.transform(data)
print(data4.head(5))

   Survived  Pclass  Sex   Age  SibSp     Fare
0         0       3    1  22.0      1   7.2500
1         1       1    2  38.0      1  71.2833
2         1       3    2  26.0      0   7.9250
3         1       1    2  35.0      1  53.1000
4         0       3    1  35.0      0   8.0500


## Mean encoding
replace the label by the mean of the target for that label. 
(the target must be 0/1 valued or continuous)


In [ ]:
# cross check-- the mean of target group by Sex
X_train['Survived'].groupby(data['Sex']).mean()


Sex
female    0.753488
male      0.196078
Name: Survived, dtype: float64

In [ ]:
mean_enc = encoding.MeanEncoding(cols=['Sex']).fit(X_train,y_train)

In [ ]:
data6 = mean_enc.transform(data)
print(data6.head(5))

   Survived  Pclass       Sex   Age  SibSp     Fare
0         0       3  0.196078  22.0      1   7.2500
1         1       1  0.753488  38.0      1  71.2833
2         1       3  0.753488  26.0      0   7.9250
3         1       1  0.753488  35.0      1  53.1000
4         0       3  0.196078  35.0      0   8.0500


## Target-encoding
Similar to mean encoding, but use both posterior probability and prior probability of the target

In [ ]:
# create the encoder and fit with our data
target_enc = ce.TargetEncoder(cols=['Sex']).fit(X_train,y_train)

In [ ]:
# perform transformation
# data.Survived.groupby(data['Sex']).agg(['mean'])
data2 = target_enc.transform(data)

In [ ]:
# check the result
data2.head()

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,0.196078,22.0,1,7.2500
1,1,1,0.753488,38.0,1,71.2833
2,1,3,0.753488,26.0,0,7.9250
3,1,1,0.753488,35.0,1,53.1000
4,0,3,0.196078,35.0,0,8.0500


## WOE-encoding
replace the label  with Weight of Evidence of each label. WOE is computed from the basic odds ratio: 

ln( (Proportion of Good Outcomes) / (Proportion of Bad Outcomes))

In [ ]:
woe_enc = ce.WOEEncoder(cols=['Sex']).fit(X_train,y_train)

In [ ]:
data3 = woe_enc.transform(data)

In [ ]:
data3.head(5)

,Survived,Pclass,Sex,Age,SibSp,Fare
0,0,3,-0.950742,22.0,1,7.2500
1,1,1,1.555633,38.0,1,71.2833
2,1,3,1.555633,26.0,0,7.9250
3,1,1,1.555633,35.0,1,53.1000
4,0,3,-0.950742,35.0,0,8.0500
